# Login HuggingFace to Upload Model

In [1]:
# !git config --global credential.helper store

# from huggingface_hub import notebook_login
# notebook_login()

# 1. Training

In [2]:
import torch
torch.cuda.empty_cache()

ModuleNotFoundError: No module named 'torch'

In [ ]:
!nvidia-smi

In [ ]:
def print_encoded(encoded_inputs):
    for ts in encoded_inputs:
        print(ts, encoded_inputs[ts])

## 1.1 Prepare-Datasets

In [ ]:
import transformers
print(transformers.__version__)

In [3]:
model_checkpoint = "Helsinki-NLP/opus-mt-th-en"

In [4]:
from datasets import load_dataset, load_metric, DatasetDict, Dataset

# raw_datasets = load_dataset("scb_mt_enth_2020", name="then")
metric = load_metric("sacrebleu")

ModuleNotFoundError: No module named 'datasets'

In [ ]:
import pandas as pd
def pre_process_from_csv(path, n_row=100000):
    df_5M = pd.read_csv(path, nrows=n_row)
    list_5M = df_5M.to_dict('records')
    list_sub = ['LST_Corpus']*len(list_5M)
    dict_5M = pd.DataFrame({"translation": list_5M, "subdataset": list_sub})
    return dict_5M

In [ ]:
raw_datasets = pre_process_from_csv('TEST_dataset_5M.csv', 1000000) # 1Mill to the top

In [ ]:
raw_datasets['translation']

In [ ]:
cut_datasets = DatasetDict()
cut_datasets = Dataset.from_pandas(raw_datasets, split="train+validation").train_test_split(0.1)

In [ ]:
cut_datasets

In [ ]:
import datasets, random
import pandas as pd
from IPython.display import display, HTML

def show_random_element(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than dataset"
    picks = []
    for n_ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks: # random inside
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_element(cut_datasets['train'])

In [ ]:
cut_datasets['train'][0:2]

#### Metric

In [ ]:
# metric

In [17]:
metric = load_metric("sacrebleu")
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

## 1.2 Preprocessing the Data

In [18]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [19]:
# Set Source and Target Language

if 'mbart' in model_checkpoint:
    tokenizer.src_lang = 'th'
    tokenizer.tgt_lang = 'en'

In [20]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    model_checkpoint = "translate Thai to English: "
else:
    prefix = ""

In [21]:
max_input_len  = 128
max_target_len = 128

source_lang = 'th'
target_lang = 'en'

def preprocess_function(examples):
#     inputs = [prefix + ex[source_lang] for ex in examples['translation']]
    inputs, targets = [], []
    for ex_ in (examples['translation']):
        ex_target = ex_[target_lang]
        if ex_target is not None:
            targets.append(ex_[target_lang])
            inputs.append(prefix + ex_[source_lang])
            
#     if len(targets) != len(inputs):
#         print(inputs)
#         print()
#         print(targets)

    model_inputs = tokenizer(inputs, max_length=max_input_len, truncation=True) # Pad to longest word (128 char)
#     print(model_inputs['input_ids'])
    
    with tokenizer.as_target_tokenizer():
        try:
            labels = tokenizer(targets, max_length=max_target_len, truncation=True)
        except:
            print('targets: ', targets)
    
    
#     if len(model_inputs['input_ids']) != len(model_inputs['input_ids']):
#         print(model_inputs)
        
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [24]:
max_input_len  = 128
max_target_len = 128

source_lang = 'th'
target_lang = 'en'

def preprocess_function(examples):
#     inputs = [prefix + ex[source_lang] for ex in examples['translation']]
    inputs, targets = [], []
    for ex_ in (examples['translation']):
        ex_target = ex_[target_lang]
        # SKIP NULL THING
        if ex_target is not None and ex_[source_lang] is not None:
            targets.append(ex_[target_lang])
            inputs.append(prefix + ex_[source_lang])
            
    model_inputs = tokenizer(inputs, max_length=max_input_len, truncation=True) # Pad to longest word (128 char)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_len, truncation=True)
    
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [25]:
tokenized_datasets = cut_datasets.map(preprocess_function, 
                                      batched=True, 
                                      batch_size=1000, 
                                      num_proc=32, 
                                      remove_columns=["translation", "subdataset"], # Prevent ArrowInvalid Error'
                                     )

In [26]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 849935
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 94312
    })
})

## 1.3 Fine-tuning the model

In [27]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [28]:
model_checkpoint

'Helsinki-NLP/opus-mt-th-en'

In [29]:
torch.cuda.empty_cache()

In [30]:
!nvidia-smi

Tue Mar  8 03:56:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1B.0 Off |                    0 |
| N/A   42C    P0    40W / 300W |      0MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:1C.0 Off |                    0 |
| N/A   44C    P0    39W / 300W |      0MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [31]:
batch_size = 32
model_name = model_checkpoint.split('/')[-1]
args = Seq2SeqTrainingArguments(
    f'{model_name}-finetuned-1Mill-{source_lang}-to-{target_lang}',
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size  = batch_size,
#     gradient_checkpointing=True, # For Accelerate Training
    weight_decay =0.01,
    save_total_limit = 3,
    num_train_epochs=1,
    predict_with_generate=True,
    dataloader_num_workers=32, # Multi-tread CPU
    fp16=True,
    eval_accumulation_steps = 10, # Reduce Using GPU Ram When Evaulation
    push_to_hub = False,
)

#### Create Data collator

In [33]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

In [34]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels= [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    print('preds', preds)
    print('labels', labels)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens = True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result['score']}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [36]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset= tokenized_datasets['train'],
#     eval_dataset = tokenized_datasets['test'],
    eval_dataset = None,
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics,
)

Using amp half precision backend


## 1.4 *train time!*

In [37]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 849935
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 6641
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


Saving model checkpoint to opus-mt-th-en-finetuned-1Mill-th-to-en/checkpoint-500
Configuration saved in opus-mt-th-en-finetuned-1Mill-th-to-en/checkpoint-500/config.json
Model weights saved in opus-mt-th-en-finetuned-1Mill-th-to-en/checkpoint-500/pytorch_model.bin
tokenizer config file saved in opus-mt-th-en-finetuned-1Mill-th-to-en/checkpoint-500/tokenizer_config.json
Special tokens file saved in opus-mt-th-en-finetuned-1Mill-th-to-en/checkpoint-500/special_tokens_map.json
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to opus-mt-th-en-finetuned-1Mill-th-to-en/checkpoint-1000
Configuration saved in opus-mt-th-en-finetuned-1Mill-th-to-en/checkpoint-1000/config.json
Model weights saved in opus-mt-th-en-finetuned-1Mill-th-to-en/checkpoint-100

ValueError: Trainer: evaluation requires an eval_dataset.

When Loading the model


``` 
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")
```



# 2. Using Translation Model

### Using AutoModel (With model selection)

In [ ]:
?AutoModelForSeq2SeqLM.

In [ ]:
# metric = load_metric("bleu")
# fake_preds = [["hello"],["there"], ["general"], ['kenobi']]
# fake_labels = [["hello"],["there"], ["general"], ['kenobi']]
# # fake_labels = [["hello there"], ["general kenobi"]]
# metric.compute(predictions=fake_preds, 
#                references=fake_labels)

In [40]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_pretrain = "Helsinki-NLP/opus-mt-th-en"
model_for_predict = "./opus-mt-th-en-finetuned-1Mill-th-to-en/checkpoint-6500"
# model_for_predict = "huak95/opus-mt-th-en-finetuned-5k-th-to-en"

model_pt = AutoModelForSeq2SeqLM.from_pretrained(model_pretrain)
model = AutoModelForSeq2SeqLM.from_pretrained(model_for_predict)
tokenizer = AutoTokenizer.from_pretrained(model_pretrain)

loading configuration file https://huggingface.co/Helsinki-NLP/opus-mt-th-en/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/0eb33d380c174e6420413d055247f46d08f0270c5b21934639530ea9cccdd756.2812763906fad9e04edf3afcd7e6da1e82df7c1856291e12132b346b18f9ee46
Model config MarianConfig {
  "_name_or_path": "Helsinki-NLP/opus-mt-th-en",
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      62306
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 62306,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "

#### PUSH TO HUB >_<

In [ ]:
!git lfs track "opus-mt-th-en-finetuned-5k-th-to-en/*"

model.push_to_hub('huak95/opus-mt-th-en-finetuned-5k-th-to-en')
# tokenizer.push_to_hub('huak95/opus-mt-th-en-finetuned-5k-th-to-en')

#### Testing >0<

In [41]:
def harry_translate(input_texts, model=model_pt):
    inputs = tokenizer(input_texts['th'], return_tensors = "pt")

    outputs = model.generate(inputs["input_ids"],
                             max_length=40,
                             num_beams=4,
                             early_stopping=True)

    return tokenizer.decode(outputs[0]).replace('<pad> ',''), input_texts['en']

In [47]:
datasets_size = len(cut_datasets['test']['translation'])
rand_idx = np.random.randint(datasets_size, size=10)

for i in rand_idx:
    print(f'index: {i}')

    text_ = cut_datasets['test']['translation'][i]
    thai_text = text_['th']
    pred_text_pt, real_text = harry_translate(text_, model_pt)
    pred_text, real_text = harry_translate(text_, model)
    
#     m_score = compute_metrics.compute(predictions=[pred_text], references=[real_text])

    print('thai_text ตัวอย่าง:\t',  thai_text)
    print('real_text เฉลย  :\t', real_text)
    print('pre_train ก่อนจูน :\t', pred_text_pt)
    print('pred_text หลังจูน :\t', pred_text)
    print()

index: 3890
thai_text ตัวอย่าง:	 เสื้อ ของ เขา ใส่ จน ขาด หมด
real_text เฉลย  :	 his coat has worn to shreds .
pre_train ก่อนจูน :	 His Clothes were torn to pieces
pred_text หลังจูน :	 his clothes were torn to pieces.

index: 41606
thai_text ตัวอย่าง:	 มี ความ ผิด ฐาน มี ส่วน ร่วม ใน ฆาตกรรม
real_text เฉลย  :	 guilty of conspiracy to murder
pre_train ก่อนจูน :	 Guilty to Participate in Murder
pred_text หลังจูน :	 be guilty to participate in the murder

index: 96276
thai_text ตัวอย่าง:	 เอกสาร ที่ มี ประโยชน์ ต่อ นัก ประวัติศาสตร์ ใน อนาคต
real_text เฉลย  :	 a document of some value to future historians
pre_train ก่อนจูน :	 A Document That Benefits Future Historians
pred_text หลังจูน :	 A Document That Benefits Future Historians

index: 80038
thai_text ตัวอย่าง:	 ความ ประทับใจ ที่ ดี
real_text เฉลย  :	 an agreeable impression
pre_train ก่อนจูน :	 A Good Impression
pred_text หลังจูน :	 a good impression

index: 87313
thai_text ตัวอย่าง:	 เป็นไปได้ มั๊ย ที่ จะเข้ า หาด ?
real_text เฉลย  :

#### VIEW NAN DATA

In [ ]:
text_df = pd.DataFrame(cut_datasets['test']['translation'])


,en,th
21,None,พวก ​ เขา ​ หยั่ง ​ รู้ ​ เข้าใจ ​ ความ ​ เร่ง...
72,None,เป็น ​ ความ ​ เบิก บาน ​ ใจ ​ ใน ​ ทุก ​ วัน ​...
81,None,เก็บ ​ รายการ ​ ข้อ ​ คัมภีร์ ​ นี้ ​ ไว้ ​ ใน...
116,None,ตัว ​ อย่าง ​ เช่น พ่อ ​ แม่ ​ บาง ​ คน ​ รู้ส...
141,None,เธอ ​ จะ ​ ให้ ​ พวก ​ เรา ​ ช่วย ​ กัน ​ จ่าย...
...,...,...
99903,None,หลัง ​ จาก ​ พระ ​ บุตร ​ สุด ​ ที่ ​ รัก ​ ขอ...
99920,None,“ ไม่ ​ ว่า ​ ฉัน ​ จะ ​ ลุก ​ หรือ ​ จะ ​ นอน...
99954,None,( ราย ​ ละเอียด ​ ดู ​ จาก ​ วารสาร )
99955,None,ผู้ ​ ที่ ​ อ้าง ​ ว่า ​ เป็น ​ คริสเตียน ​ ได...


In [89]:
datasets_size = len(cut_datasets['test']['translation'])
rand_idx = np.random.randint(datasets_size, size=100)

datasets_small = text_df[text_df.en.isnull()]
for i in (datasets_small).index:
    print(f'index: {i}')


    thai_text = datasets_small['th'].loc[i]
    real_text = datasets_small['en'].loc[i]
#     pred_text_pt, real_text = harry_translate(text_, model_pt)
#     pred_text, real_text = harry_translate(text_, model)
    
#     m_score = compute_metrics.compute(predictions=[pred_text], references=[real_text])

    print('thai_text ตัวอย่าง:\t',  thai_text)
    print('real_text เฉลย  :\t', real_text)
#     print('pre_train ก่อนจูน :\t', pred_text_pt)
#     print('pred_text หลังจูน :\t', pred_text)
    print()

index: 21
thai_text ตัวอย่าง:	 พวก ​ เขา ​ หยั่ง ​ รู้ ​ เข้าใจ ​ ความ ​ เร่ง ​ ด่วน ​ ของ ​ งาน ​ นี้ และ ​ ต้องการ ​ มี ​ ส่วน ​ อย่าง ​ เต็ม ​ ที่ ​ ที่ ​ สุด ​ เท่า ​ ที่ ​ เป็น ​ ไป ​ ได้ ​ เพื่อ ​ ทำ ​ ให้ ​ งาน ​ นี้ ​ สำเร็จ ​ ก่อน ​ อวสาน ​ ของ ​ โลก ​ ที่ ​ ไม่ ​ เป็น ​ สุข ​ นี้ ​ มา ​ ถึง .
real_text เฉลย  :	 None

index: 72
thai_text ตัวอย่าง:	 เป็น ​ ความ ​ เบิก บาน ​ ใจ ​ ใน ​ ทุก ​ วัน ​ นี้ ​ ที่ ​ แล ​ เห็น ​ บุตร ​ จำนวน ​ ไม่ ​ น้อย ​ ของ ​ พยาน ​ พระ ​ ยะ โฮ วา ​ ปฏิบัติ ​ ตาม ​ คำ ​ สอน ​ ดัง ​ กล่าว โดย ​ การ ​ ใช้ ​ ชีวิต ​ วัย รุ่น ​ ของ ​ ตน ​ เตรียม ​ ตัว ​ เพื่อ ​ ทำ ​ งาน ​ รับ ใช้ ​ พระ ​ ยะ โฮ วา ​ เต็ม ​ เวลา และ ​ เมื่อ ​ จบ ​ โรง ​ เรียน ​ แล้ว ​ ก้าว ​ เข้า ​ สู่ ​ งาน ​ ประจำ ​ ชีวิต ​ ที่ ​ ยิ่ง ​ ใหญ่ ​ ที่ ​ สุด .
real_text เฉลย  :	 None

index: 81
thai_text ตัวอย่าง:	 เก็บ ​ รายการ ​ ข้อ ​ คัมภีร์ ​ นี้ ​ ไว้ ​ ใน ​ ที่ ​ ที่ ​ เห็น ​ ได้ ​ สะดวก และ ​ อ่าน ​ บ่อย ๆ .
real_text เฉลย  :	 None

index: 116
thai_text ตัวอย่าง:	 ตัว ​ อย่าง ​ เช่น พ่อ

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
datasets_small['len_th'] = datasets_small['th']

In [90]:
datasets_small['th']

21       พวก ​ เขา ​ หยั่ง ​ รู้ ​ เข้าใจ ​ ความ ​ เร่ง...
72       เป็น ​ ความ ​ เบิก บาน ​ ใจ ​ ใน ​ ทุก ​ วัน ​...
81       เก็บ ​ รายการ ​ ข้อ ​ คัมภีร์ ​ นี้ ​ ไว้ ​ ใน...
116      ตัว ​ อย่าง ​ เช่น พ่อ ​ แม่ ​ บาง ​ คน ​ รู้ส...
141      เธอ ​ จะ ​ ให้ ​ พวก ​ เรา ​ ช่วย ​ กัน ​ จ่าย...
                               ...                        
99903    หลัง ​ จาก ​ พระ ​ บุตร ​ สุด ​ ที่ ​ รัก ​ ขอ...
99920    “ ไม่ ​ ว่า ​ ฉัน ​ จะ ​ ลุก ​ หรือ ​ จะ ​ นอน...
99954                ( ราย ​ ละเอียด ​ ดู ​ จาก ​ วารสาร )
99955    ผู้ ​ ที่ ​ อ้าง ​ ว่า ​ เป็น ​ คริสเตียน ​ ได...
99978    เอียน — การ ​ เป็น ​ ลูก ​ ผู้ ​ ชาย ​ ไม่ ​ ไ...
Name: th, Length: 5300, dtype: object